In [ ]:
import pandas as pd
import rasterio
from rasterio.sample import sample_gen
from matplotlib import pyplot

In [ ]:
# Change to path where you store the dataset
df = pd.read_csv("~/Downloads/BOLD_Public.19-Apr-2024.Amazon_Countries.tsv", sep="\t", on_bad_lines="skip")

# Change to the respective tif file
ecolayer = "~/Downloads/CHELSA_BIO_Annual_Mean_Temperature.tif"

def parse_coords(df):
    coords = []
    for coord in df["coord"]:
        parts = coord[1:-1].split(",")
        x, y = float(parts[0]), float(parts[1])

        coords.append((y, x))

    return coords

with rasterio.open(ecolayer) as dataset:
    extent = [dataset.bounds[0], dataset.bounds[2], dataset.bounds[1], dataset.bounds[3]]
    # This shows the geographical coordinates of the region of interest
    print(extent)

    # Render the eclogical layer's first channel
    array = dataset.read(1)
    pyplot.imshow(array, cmap='pink')
    pyplot.show()

    # From the eDNA dataset only get the samples which are within the region of interest
    inbound_coords = []
    coords = parse_coords(df)
    for x, y in coords:
        if x >= extent[0] and x <= extent[1] and y >= extent[2] and y <= extent[3]:
            inbound_coords.append((x, y))

    print(inbound_coords)
    print(len(inbound_coords))

    # Get the corresponding ecological values for the samples
    results = sample_gen(dataset, inbound_coords)
    results = [r for r in results]

    print(results)